<a href="https://colab.research.google.com/github/ak04vb06/lab-sql-query-from-table-names/blob/main/lab_sql_generation_with_transformer_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Generation with Transformer API

In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 13.3 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
print("Libraries imported successfully!")

Libraries imported successfully!


In [ ]:
torch.cuda.is_available()

True

In [ ]:
import torch

if torch.cuda.is_available():
    available_memory = torch.cuda.get_device_properties(0).total_memory
    print(f"GPU Memory: {available_memory / 1024**3:.2f} GB")
else:
    available_memory = 0
    print("No GPU found. Using CPU (this will be slow).")

GPU Memory: 14.56 GB


In [ ]:
print(available_memory)

15835660288

##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [ ]:
Zmodel_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [ ]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [ ]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [ ]:
question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question)

In [ ]:
print(generated_sql)


SELECT p.product_id,
       SUM(s.quantity * p.price) AS revenue
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.product_id
ORDER BY revenue DESC NULLS LAST;


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# Report: SQL Generation with Transformer API

## Summary of Findings

In this exercise, we tested the `defog/sqlcoder-7b-2` model's ability to generate SQL queries from natural language questions based on a provided schema. We tested three distinct scenarios:

1.  **Aggregation and Filtering (Salespeople Revenue):**
    -   **Question:** "Show me the top 5 salespeople by total revenue generated in 2023."
    -   **Observation:** The model correctly identified the need to join `salespeople`, `sales`, and `products` tables. It successfully calculated revenue as `price * quantity`, filtered by the year 2023 using `EXTRACT(YEAR FROM sale_date)`, and applied `ORDER BY` and `LIMIT` clauses correctly.

2.  **Complex Filtering with Multiple Joins (Customer Supply Price):**
    -   **Question:** "List all customers who bought products supplied by suppliers with a supply price greater than 50."
    -   **Observation:** The model successfully navigated a four-table join (`customers` -> `sales` -> `products` -> `product_suppliers`). It correctly applied the filter `supply_price > 50`. The selection of columns was logical, providing context (names) rather than just IDs.

3.  **Grouping and Aggregation (Regional Sales Quantity):**
    -   **Question:** "What is the total quantity of products sold by each salesperson in the 'West' region?"
    -   **Observation:** The model correctly filtered by `region = 'West'` on the `salespeople` table and joined it with `sales`. It aggregated `quantity` by `salesperson_id`.

## Conclusion & Learnings

The model demonstrated a strong understanding of the provided schema and standard SQL syntax (PostgreSQL dialect).

-   **Schema Adherence:** It respected the foreign key relationships described in the prompt.
-   **Calculation Logic:** It correctly interpreted "revenue" as price times quantity, a rule explicitly provided in the prompt instructions.
-   **Hallucinations:** No obvious hallucinations were observed in these three examples; column names and table names matched the schema.

Overall, the prompt engineering strategy of providing the schema definition and specific business rules (like revenue calculation) proved effective for this model.

In [16]:
question_1 = "Show me the top 5 salespeople by total revenue generated in 2023."
generated_sql_1 = generate_query(question_1)
print(f"Question 1: {question_1}\n")
print(generated_sql_1)

Question 1: Show me the top 5 salespeople by total revenue generated in 2023.


SELECT sp.name,
       SUM(p.price * s.quantity) AS total_revenue
FROM salespeople sp
JOIN sales s ON sp.salesperson_id = s.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE EXTRACT(YEAR
              FROM s.sale_date) = 2023
GROUP BY sp.name
ORDER BY total_revenue DESC NULLS LAST
LIMIT 5;


In [17]:
question_2 = "List all customers who bought products supplied by suppliers with a supply price greater than 50."
generated_sql_2 = generate_query(question_2)
print(f"Question 2: {question_2}\n")
print(generated_sql_2)

Question 2: List all customers who bought products supplied by suppliers with a supply price greater than 50.


SELECT c.name AS customer_name,
       p.name AS product_name,
       ps.supply_price AS supply_price,
       s.quantity AS quantity_sold
FROM customers c
JOIN sales s ON c.customer_id = s.customer_id
JOIN products p ON s.product_id = p.product_id
JOIN product_suppliers ps ON p.product_id = ps.product_id
WHERE ps.supply_price > 50;


In [18]:
question_3 = "What is the total quantity of products sold by each salesperson in the 'West' region?"
generated_sql_3 = generate_query(question_3)
print(f"Question 3: {question_3}\n")
print(generated_sql_3)

Question 3: What is the total quantity of products sold by each salesperson in the 'West' region?


SELECT s.salesperson_id,
       SUM(s.quantity) AS total_quantity
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
WHERE sp.region = 'West'
GROUP BY s.salesperson_id
ORDER BY total_quantity DESC NULLS LAST;
